In [1]:
# This changes the current directory to the base saga directory - make sure to run this first!
# This is necessary to be able to import the py files and use the right directories,
# while keeping all the notebooks in their own directory.
import os
import sys

if 'saga_base_dir' not in locals():
    saga_base_dir = os.path.abspath('..')

if saga_base_dir not in sys.path:
    os.chdir(saga_base_dir)

In [2]:
from __future__ import print_function, division

import numpy as np

import hosts
import targeting

from astropy import units as u
from astropy.coordinates import *
from astropy import table
from astropy.visualization import hist as ahist
from astropy.io import fits

In [3]:
%matplotlib inline
from matplotlib import pyplot as plt
from matplotlib import rcParams

rcParams['image.interpolation'] = 'none'
rcParams['figure.figsize'] = (16, 10)

In [4]:
def add_coord_to_tab(tab, raname='RA', decname='Dec', unit=u.deg, coocolname='skycoord'):
    coo = SkyCoord(ra=tab[raname], dec=tab[decname], unit=unit, frame='icrs')
    tab[coocolname] = coo
    return coo

# Load the SAGA datasets 

## First load the flag0/host lists

In [5]:
export_url_templ = 'https://docs.google.com/spreadsheets/d/{id}/export?format=csv&id={id}&gid={gid}'
base_id = '1b3k2eyFjHFDtmHce1xi6JKuj3ATOWYduTBFftx5oPp8'
flag0_gid = '448084634'
nsats_gid = '271797930'


tab_flag0 = table.Table.read(export_url_templ.format(id=base_id, gid=flag0_gid), format='csv')
tab_nsats0 = table.Table.read(export_url_templ.format(id=base_id, gid=nsats_gid), format='csv')

In [6]:
print(len(tab_flag0))
tab_flag0[:5].show_in_notebook()

74


NSAID,RA,Dec,PGC#,vhelio,distance,r,i,z,I,K,K_err,flag,K_abs,r_abs,b,dist_to_group,brightest_K_nearby,SDSS_FRACGOOD,u-r,sersic_n,stellar_mass,N_sats_all,N_sats_removed,N_sats_rem1and2,N_SDSS_sats,N_sats_total
145729,224.594,-1.09102,53499,1922.0,28.4038,11.9155,11.457,11.0653,9.49,7.693,0.019,0,-24.5739,-20.3514,48.4266,3.55266,12.164,0.983605,2.6946,2.82806,19800000000.0,1,1,1,1,1
150887,348.683,4.53406,70795,2689.0,37.019,11.9764,11.6247,11.3038,10.57,8.327,0.013,0,-24.5151,-20.8657,-50.6545,5.73166,10.189,0.98742,1.90022,0.777359,25100000000.0,0,0,0,0,0
140594,180.098,-1.10008,37845,1465.0,23.2487,11.1116,10.7406,10.4707,9.35,7.321,0.015,0,-24.511,-20.7204,59.2146,2.29212,12.669,0.985217,2.05631,1.7545,20600000000.0,2,1,2,1,1
142722,194.863,14.1712,44600,1988.0,30.027,11.4723,11.0534,10.7367,9.93,7.907,0.016,0,-24.4806,-20.9153,76.9088,3.38499,10.768,0.987676,2.70741,2.18747,35000000000.0,5,2,2,2,2
150578,346.236,12.3229,70419,2381.0,33.313,11.5503,11.1597,10.903,9.89,8.162,0.021,0,-24.4511,-21.0628,-42.8417,9.44227,14.017,0.987084,2.07284,2.01064,34400000000.0,0,0,0,0,0


In [7]:
print(len(tab_nsats0))
tab_nsats0[:5].show_in_notebook()

74


NSAID,RA,Dec,N_sats_all,N_sats_removed,N_sats_rem1and2,N_SDSS_sats,N_sats_total
145729,224.594,-1.09102,1,1,1,1,1
150887,348.683,4.53406,0,0,0,0,0
140594,180.098,-1.10008,2,1,2,1,1
142722,194.863,14.1712,5,2,2,2,2
150578,346.236,12.3229,0,0,0,0,0


In [8]:
# double-check that the above two are aligned
assert np.all(tab_flag0['NSAID']==tab_nsats0['NSAID'])

### Combine the two together 

In [9]:
tab_hosts = table.join(tab_flag0, tab_nsats0[[nm for nm in tab_nsats0.colnames if nm.startswith('N')]], 'NSAID')
add_coord_to_tab(tab_hosts)

print(len(tab_hosts))
tab_hosts[:5].show_in_notebook()

74


NSAID,RA,Dec,PGC#,vhelio,distance,r,i,z,I,K,K_err,flag,K_abs,r_abs,b,dist_to_group,brightest_K_nearby,SDSS_FRACGOOD,u-r,sersic_n,stellar_mass,N_sats_all_1,N_sats_removed_1,N_sats_rem1and2_1,N_SDSS_sats_1,N_sats_total_1,N_sats_all_2,N_sats_removed_2,N_sats_rem1and2_2,N_SDSS_sats_2,N_sats_total_2,skycoord
,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,"deg,deg"
32,145.514,0.336452,27723,1912.35,30.0374,11.8626,11.603,11.4285,10.53,8.848,0.036,0,-23.5403,-20.5257,37.2791,8.62942,13.304,0.974112,1.63536,1.3137,16800000000.0,1,1,1,1,1,1,1,1,1,1,"145.514,0.336452"
13927,180.676,1.9768,38031,1961.14,29.5168,12.0011,11.5644,11.2205,10.98,8.743,0.027,0,-23.6073,-20.3492,62.2704,3.36159,11.815,0.98222,2.43461,3.12889,19800000000.0,0,0,0,0,0,0,0,0,0,0,"180.676,1.9768"
16235,139.416,52.993,26246,2274.04,36.2141,12.9972,12.5874,12.277,14.99,9.406,0.02,0,-23.3884,-19.7972,43.0944,13.055,12.621,0.972638,2.35828,1.49728,12000000000.0,8,3,3,3,3,8,3,3,3,3,"139.416,52.993"
16559,154.888,58.2058,30176,2099.41,34.4951,12.3762,11.8854,11.586,11.85,9.469,0.031,0,-23.2198,-20.3126,49.2187,8.48479,11.148,0.98321,1.98864,1.75007,19100000000.0,4,2,3,3,3,4,2,3,3,3,"154.888,58.2058"
33446,123.241,36.2546,23028,2469.7,37.7379,12.3668,12.0137,11.739,11.77,9.407,0.032,0,-23.4769,-20.5171,31.306,14.5285,12.875,0.972509,2.04922,2.13367,21200000000.0,2,2,2,2,2,2,2,2,2,2,"123.241,36.2546"


## And also load the sats list 

In [10]:
tab_specs = table.Table.read('SAGADropbox/data/saga_spectra_clean.fits.gz')
add_coord_to_tab(tab_specs, raname='RA', decname='DEC')

tab_sats = tab_specs[(tab_specs['ZQUALITY']>=3)&(tab_specs['REMOVE']==-1)&(tab_specs['SATS']==1)]

print(len(tab_specs), len(tab_sats))
tab_sats[:5].show_in_notebook()

48238 137


OBJID,RA,DEC,PHOTPTYPE,PHOT_SG,FLAGS,SATURATED,BAD_COUNTS_ERROR,BINNED1,u,g,r,i,z,u_err,g_err,r_err,i_err,z_err,MODELMAGERR_U,MODELMAGERR_G,MODELMAGERR_R,MODELMAGERR_I,MODELMAGERR_Z,EXTINCTION_U,EXTINCTION_G,EXTINCTION_R,EXTINCTION_I,EXTINCTION_Z,DERED_U,DERED_G,DERED_R,DERED_I,DERED_Z,PETRORAD_U,PETRORAD_G,PETRORAD_R,PETRORAD_I,PETRORAD_Z,PETRORADERR_U,PETRORADERR_G,PETRORADERR_R,PETRORADERR_I,PETRORADERR_Z,DEVRAD_U,DEVRADERR_U,DEVRAD_G,DEVRADERR_G,DEVRAD_R,DEVRADERR_R,DEVRAD_I,DEVRADERR_I,DEVRAD_Z,DEVRADERR_Z,DEVAB_U,DEVAB_G,DEVAB_R,DEVAB_I,DEVAB_Z,CMODELMAG_U,CMODELMAGERR_U,CMODELMAG_G,CMODELMAGERR_G,CMODELMAG_R,CMODELMAGERR_R,CMODELMAG_I,CMODELMAGERR_I,CMODELMAG_Z,CMODELMAGERR_Z,PSFMAG_U,PSFMAGERR_U,PSFMAG_G,PSFMAGERR_G,PSFMAG_R,PSFMAGERR_R,PSFMAG_I,PSFMAGERR_I,PSFMAG_Z,PSFMAGERR_Z,FIBERMAG_U,FIBERMAGERR_U,FIBERMAG_G,FIBERMAGERR_G,FIBERMAG_R,FIBERMAGERR_R,FIBERMAG_I,FIBERMAGERR_I,FIBERMAG_Z,FIBERMAGERR_Z,FRACDEV_U,FRACDEV_G,FRACDEV_R,FRACDEV_I,FRACDEV_Z,Q_U,U_U,Q_G,U_G,Q_R,U_R,Q_I,U_I,Q_Z,U_Z,EXPAB_U,EXPRAD_U,EXPPHI_U,EXPAB_G,EXPRAD_G,EXPPHI_G,EXPAB_R,EXPRAD_R,EXPPHI_R,EXPAB_I,EXPRAD_I,EXPPHI_I,EXPAB_Z,EXPRAD_Z,EXPPHI_Z,FIBER2MAG_R,FIBER2MAGERR_R,EXPMAG_R,EXPMAGERR_R,Column1,PETROR50_R,PETROR90_R,PETROMAG_R,SB_EXP_R,SB_PETRO_R,J,JERR,H,HERR,K,KERR,SPEC_Z,SPEC_Z_ERR,SPEC_Z_WARN,PHOTOZ,PHOTOZ_ERR,W1,W1ERR,W2,W2ERR,HOST_RA,HOST_DEC,HOST_DIST,HOST_VHOST,HOST_MK,HOST_NSAID,HOST_FLAG,HOST_SAGA_NAME,RHOST_ARCM,RHOST_KPC,OBJ_NSAID,SATS,PROBABILITY_CLASS1,RESCALED_PROBABILITY_CLASS1,REMOVE,TELNAME,MASKNAME,ZQUALITY,SPEC_CLASS_SGQ,SPEC_CLASS_AE,SPECOBJID,SPEC_REPEAT,skycoord
,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,"deg,deg"
1237655693015056396,224.532617519,-1.31259609568,3,GALAXY,35253360398352,262144,0,268435456,17.3497,16.089,15.4546,15.1464,14.9615,0.0416732,0.00498512,0.00534471,0.00545955,0.0135447,0.0416732,0.00498512,0.00534471,0.00545955,0.0135447,0.32708,0.240662,0.174548,0.132355,0.0938412,17.1418,15.8741,15.3876,15.0828,14.9173,12.9075,13.4682,13.1628,13.8231,13.8238,0.738636,0.710056,0.803463,0.78797,0.894669,21.15,3.11296,24.4211,0.411017,23.5287,0.513989,23.8198,0.408222,24.1926,1.04576,0.449992,0.465957,0.446009,0.475622,0.467311,17.477,0.0415375,16.0842,0.00498177,15.5622,0.00534471,15.1929,0.00545646,15.0088,0.0135404,21.1691,0.0788425,19.6741,0.0184681,19.4228,0.0195905,19.0933,0.019701,18.8251,0.037165,20.6698,0.0618124,19.3683,0.0111833,18.7532,0.0087296,18.4167,0.00977192,18.174,0.0217943,0.0,0.0,0.0,0.0,0.0,0.0862015,-0.015842,0.139541,-0.0327954,0.14517,-0.0385242,0.135794,-0.034536,0.133919,-0.0334079,0.508928,8.21412,149.518,0.479386,8.81146,151.674,0.473689,8.55776,152.901,0.487392,8.70547,151.876,0.495454,8.42529,152.167,19.5874,0.0125185,15.5622,0.00534471,8.55776,6.02391,13.3899,15.5966,22.2194214691,21.4913916271,15.174,9999.0,15.71,0.248,14.45,9999.0,0.0064927,2.15047e-05,0,0.067474,0.016437,-1.0,-1.0,-1.0,-1.0,224.594,-1.09102,28.4038,1922.0,-24.5738822299,145729,0,,13.7934861167,113.96603527,35536,1,0.559135409723,0.691219228277,-1,NSA,sdss,4,,,1.23765569302e+18,NSA,"224.532617519,-1.31259609568"
1237648702984683605,225.005401226,-1.09130226734,3,GALAXY,1161121054724696,0,0,268435456,16.0611,15.3154,14.9987,14.8723,14.7438,0.00790307,0.00274512,0.00320407,0.00445661,0.00904204,0.00790307,0.00274512,0.00320407,0.00445661,0.00904204,0.327712,0.241128,0.174886,0.132611,0.0940226,15.997,15.1689,14.8268,14.7113,14.6349,40.986,12.9685,13.7948,14.3383,13.7471,1.52479,0.171283,0.158666,0.162538,0.28877,10.7435,0.29789,14.1078,0.139925,15.0666,0.189859,16.1199,0.217477,19.1789,0.55349,0.763178,0.837018,0.889757,0.903721,0.889412,16.1489,0.00772101,15.3572,0.00280809,14.9363,0.00327976,14.7643,0.00451928,14.6683,0.00921106,18.9691,0.0220671,18.7247,0.0217491,18.3088,0.0183036,18.6214,0.0184466,18.4012,0.0271797,18.6917,0.0133083,18.1511,0.00473336,17.9322,0.00489516,17.876,0.00606622,1

### Also figure out which are the saga hosts

In [11]:
saga_hosts = hosts.get_saga_hosts_from_google()
saga_hosts_nsaids = {host.nsaid: host for host in saga_hosts}

Using cached version of google hosts list from file "hosts_dl.pkl2"


In [12]:
saga_names = []
for nsaid in tab_hosts['NSAID']:
    if nsaid in saga_hosts_nsaids:
        saga_names.append(saga_hosts_nsaids[nsaid].name)
    else:
        saga_names.append('')
tab_hosts['SAGA_name'] = saga_names

# Now collect Jason T's QSO datasets 

In [13]:
tab_qso = table.Table.read('catalogs/dr7qso_galex.fits')
add_coord_to_tab(tab_qso, raname='RA', decname='DEC')

tab_qso19 = tab_qso[(0<tab_qso['FUV_MAG'])&(tab_qso['FUV_MAG']<19)]

print(len(tab_qso), len(tab_qso19))
tab_qso19[:5].show_in_notebook()

105811 2627


SDSSNAME,RA,DEC,Z,PSFMAG_U,PSFMAGERR_U,PSFMAG_G,PSFMAGERR_G,PSFMAG_R,PSFMAGERR_R,PSFMAG_I,PSFMAGERR_I,PSFMAG_Z,PSFMAGERR_Z,A_U,LOGNHI,FIRSTMAG,FIRST_SN,FIRST_SEP,LG_RASS_RATE,RASS_SN,RASS_SEP,TWOMASSMAG_J,TWOMASSMAGERR_J,TWOMASSMAG_H,TWOMASSMAGERR_H,TWOMASSMAG_K,TWOMASSMAGERR_K,TWOMASS_SEP,M_I,DELTA_G_I,MORPHOLOGY,SCIENCEPRIMARY,MODE,UNIFORM,BESTPRIMTARGET,TS_B_QSO,TS_B_HIZ,TS_B_FIRST,TS_B_ROSAT,TS_B_SERENDIP,TS_B_STAR,TS_B_GAL,RUN_BEST,MJD_BEST,MJD_SPEC,PLATE,FIBERID,RERUN_BEST,CAMCOL_BEST,FIELD_BEST,OBJ_BEST,TARGPRIMTARGET,TS_T_QSO,TS_T_HIZ,TS_T_FIRST,TS_T_ROSAT,TS_T_SERENDIP,TS_T_STAR,TS_T_GAL,T_PSFMAG_U,T_PSFMAGERR_U,T_PSFMAG_G,T_PSFMAGERR_G,T_PSFMAG_R,T_PSFMAGERR_R,T_PSFMAG_I,T_PSFMAGERR_I,T_PSFMAG_Z,T_PSFMAGERR_Z,BESTOBJID,SPECOBJID,OBJECT_NAME,GALEX_OBJID,GALEX_RA,GALEX_DEC,GALEX_SEP,GALEX_TYPE,GALEX_BAND,NUV_FLUX,NUV_FLUXERR,FUV_FLUX,FUV_FLUXERR,NUV_MAG,NUV_MAGERR,FUV_MAG,FUV_MAGERR,GALEX_NOBJ,skycoord
,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,"deg,deg"
000011.96+000225.3,0.0498390011489,0.0403649993241,0.4789,18.228,0.027,17.934,0.02,17.995,0.019,17.934,0.014,17.873,0.029,0.161,20.502,0.0,0.0,0.0,-1.66,2.157,2.04,16.651,0.136,15.82,0.149,14.821,0.111,0.209,-24.068,-0.146,0,1,1,0,134224898,1,0,0,1,0,0,0,3325,52522,51791,387,200,41,4,108,19,134224900,1,0,0,1,0,0,0,17.835,0.011,17.58,0.004,17.695,0.005,17.646,0.006,17.66,0.024,588015509269381139,109153257477111808,LBQS 2357-0014,6381013764764861047,0.0498250763621,0.0404867856438,0.441287278442,1,3,198.586,9.1663,134.936,12.7213,18.1527,0.0501274,18.5776,0.102384,2,"0.0498390011489,0.0403649993241"
000110.97-105247.4,0.295722007751,-10.8798570633,0.5282,17.999,0.022,17.674,0.026,17.646,0.016,17.484,0.024,17.512,0.028,0.183,20.439,0.0,0.0,0.0,-9.0,0.0,0.0,16.467,0.095,15.614,0.112,14.916,0.108,0.155,-24.764,0.017,0,1,1,0,1048578,1,0,0,0,1,0,0,1740,51814,52143,650,126,40,1,24,71,1048580,1,0,0,0,1,0,0,17.95,0.018,17.65,0.018,17.658,0.018,17.506,0.018,17.501,0.021,587727225153388615,183182687870124032,SDSS J000110.97-105247.4,2435007175675876212,0.295777766876,-10.8798445586,0.20219960592,0,3,226.309,2.69221,141.979,3.5783,18.0108,0.0129192,18.5223,0.0273704,2,"0.295722007751,-10.8798570633"
000410.80-104527.1,1.04502701759,-10.7575368881,0.2397,18.125,0.025,17.951,0.02,17.681,0.027,17.531,0.013,17.432,0.021,0.192,20.444,0.0,0.0,0.0,-1.352,3.3,7.26,16.097,0.087,15.181,0.098,14.368,0.072,0.091,-22.829,-0.221,0,1,1,0,7170,1,0,0,1,0,0,0,1729,51813,52143,650,18,40,2,90,17,7172,1,0,0,1,0,0,0,18.096,0.018,17.929,0.018,17.693,0.017,17.522,0.013,17.409,0.018,587727178449944593,183182687417139200,SDSS J000410.80-104527.1,6380415564941101376,1.04514692228,-10.7574263506,0.581539875822,1,3,101.836,6.96794,113.507,11.1903,18.8778,0.0743079,18.7653,0.107066,2,"1.04502701759,-10.7575368881"
000548.24-084808.4,1.45100700855,-8.80234718323,1.1906,17.968,0.034,17.874,0.035,17.623,0.019,17.636,0.022,17.715,0.027,0.172,20.487,0.0,0.0,0.0,-9.0,0.0,0.0,17.126,0.259,16.657,0.296,0.0,0.0,0.019,-26.577,-0.057,0,1,1,0,2,1,0,0,0,0,0,0,1740,51814,52141,651,328,40,6,32,7,4,1,0,0,0,0,0,0,17.92,0.014,17.892,0.014,17.663,0.007,17.629,0.018,17.705,0.025,587727227838267399,183464155104149504,SDSS J000548.24-084808.4,6380415545613749966,1.45083866643,-8.8023579513,0.600147137654,1,3,221.995,7.46231,98.0474,8.59181,18.0317,0.0365057,18.9243,0.0951652,2,"1.45100700855,-8.80234718323"
000557.23+002837.7,1.48849904537,0.477142989635,0.2596,18.587,0.024,18.566,0.024,18.41,0.023,18.369,0.027,17.857,0.049,0.282,20.479,0.0,0.0,0.0,-9.0,0.0,0.0,16.691,0.149,15.911,0.139,15.121,0.106,0.125,-22.215,-0.202,0,1,1,0,3,1,1,0,0,0,0,0,3325,52522,51793,388,385,41,5,118,37,4,1,0,0,0,0,0,0,18.517,0.018,18.493,0.008,18.303,0.011,18.173,0.014,17.795,0.023,588015509806907429,109434741819703296,LBQS 0003+0011,6381013763691121033,1.48841064477,0.477408398602,1.00707414717,1,3,98.6517,6.81361,123.574,11.0682,18.9123,0.0750071,18.6731,0.0972697,1,"1.48849904537,0.477142989635"


In [14]:
with fits.open('https://archive.stsci.edu/missions/hlsp/uvqs/hlsp_uvqs_multi_multi_all_multi_v1_redshifts.fits', 
              cache=True) as f:
    uvqs_tab = table.Table.read(f)
add_coord_to_tab(uvqs_tab, raname='RA', decname='DEC')

# JT recommended this cut
uvqs_good_tab = uvqs_tab[uvqs_tab['Z_QUAL']>=3]

print(len(uvqs_good_tab))
uvqs_good_tab[:5].show_in_notebook()

1121


DEC,FUV,NAME,NUV,OBJ,OBJ_IDX,OBJ_IDX2,OBSV,OBS_DATE,RA,SCI_FILE,SCND_FILE,SPEC_QUAL,THETA [20],W1,W2,Z,ZASS_FIL,Z_QUAL,Z_SIG,comment,dbx_path,instr,CAND,SPEC_FILE,FINDER,skycoord
deg,,,,,,,,,deg,,,,,,,,,,,,,,,,,"deg,deg"
-20.0743611111,18.2729988098,UVQSJ000000.15-200427.7,17.9699993134,0,1,-1,LCO,Aug2014,0.000625,09aug2014/Science/sci-ccd0918.fits.gz,,3,4.39948940277 .. 0.0,13.5480003357,12.5389995575,0.290799587965,LCO_09aug2014_zassess.fits,4,0.000171106687048,,LCO/August2014,BCS,1,UVQSJ000000.15-200427.7_LCO.fits,UVQSJ000000.15-200427.7_DSS.pdf,"0.000625,-20.0743611111"
-39.2966388889,17.8244285583,UVQSJ000503.70-391747.9,17.2321929932,0,0,-1,LCO,Aug2014,1.26541666667,07aug2014/Science/sci-ccd0645.fits.gz,,3,8.0866317749 .. 0.0,12.2600002289,11.1239995956,0.652394294739,LCO_07aug2014_zassess.fits,3,0.000363261176972,,LCO/August2014,BCS,1,UVQSJ000503.70-391747.9_LCO.fits,UVQSJ000503.70-391747.9_DSS.pdf,"1.26541666667,-39.2966388889"
-26.1945833333,18.1590518951,UVQSJ000609.57-261140.5,17.5298900604,0,1,-1,LCO,Aug2014,1.539875,09aug2014/Science/sci-ccd0920.fits.gz,,3,5.21646022797 .. 0.0,13.3079996109,12.1239995956,0.648431241512,LCO_09aug2014_zassess.fits,3,0.000249826902291,,LCO/August2014,BCS,1,UVQSJ000609.57-261140.5_LCO.fits,UVQSJ000609.57-261140.5_DSS.pdf,"1.539875,-26.1945833333"
42.2796111111,18.0854568481,UVQSJ000717.69+421646.6,17.6127948761,0,0,0,Lick,Jan2015,1.82370833333,Jan_2015/Science/sci-b337.fits.gz,Jan_2015/Science/sci-r337.fits.gz,4,17.7751274109 .. 0.0,12.4420003891,11.5080003738,0.0,Lick_jan2015_zassess.fits,4,0.000228934150073,,Lick,Kast,1,UVQSJ000717.69+421646.6_Lick.fits,UVQSJ000717.69+421646.6_DSS.pdf,"1.82370833333,42.2796111111"
-63.8627222222,17.9553394318,UVQSJ000741.00-635145.8,17.4110488892,0,0,-1,LCO,Aug2014,1.92083333333,07aug2014/Science/sci-ccd0640.fits.gz,,3,4.41201686859 .. 0.0,12.6529998779,11.454000473,0.55930185318,LCO_07aug2014_zassess.fits,3,0.000405291531933,,LCO/August2014,BCS,1,UVQSJ000741.00-635145.8_LCO.fits,UVQSJ000741.00-635145.8_DSS.pdf,"1.92083333333,-63.8627222222"


# X-match hosts 

In [15]:
def get_host_matches(hosttab, qsotab, rvir=300*u.kpc):
    tabs = []
    for row in hosttab:
        sc = row['skycoord']
        rvirsep = (rvir/(row['distance']*u.Mpc)).decompose()*u.radian
        seps = sc.separation(qsotab['skycoord'])
        
        subtab = qsotab[seps < rvirsep]
        subtab['qso_to_host'] = seps[seps < rvirsep]
        subtab['qso_to_host_phys'] = seps[seps < rvirsep].to(u.radian).value*(row['distance']*u.Mpc).to(u.kpc)

        tabs.append(subtab)
    return tabs

In [16]:
host_matches = get_host_matches(tab_hosts, tab_qso)
host_matches19 = get_host_matches(tab_hosts, tab_qso19)

In [17]:
n19 = [np.sum((0<match['FUV_MAG'])&(match['FUV_MAG']<19)) for match in host_matches]
n185 = [np.sum((0<match['FUV_MAG'])&(match['FUV_MAG']<18.5)) for match in host_matches]
n18 = [np.sum((0<match['FUV_MAG'])&(match['FUV_MAG']<18)) for match in host_matches]

tab_hosts['n_qso_fuv_lt_19'] = n19
tab_hosts['n_qso_fuv_lt_18.5'] = n185
tab_hosts['n_qso_fuv_lt_18'] = n18

In [18]:
host_matches_uvqs = get_host_matches(tab_hosts, uvqs_good_tab)

In [19]:
n19_uvqs = [np.sum((0<match['FUV'])&(match['FUV']<19)) for match in host_matches_uvqs]
n185_uvqs = [np.sum((0<match['FUV'])&(match['FUV']<18.5)) for match in host_matches_uvqs]
n18_uvqs = [np.sum((0<match['FUV'])&(match['FUV']<18)) for match in host_matches_uvqs]

tab_hosts['n_uvqs_fuv_lt_19'] = n19_uvqs
tab_hosts['n_uvqs_fuv_lt_18.5'] = n185_uvqs
tab_hosts['n_uvqs_fuv_lt_18'] = n18_uvqs

## Some summaries of numbers for hosts 

In [20]:
tab_hosts.show_in_notebook(75)

NSAID,RA,Dec,PGC#,vhelio,distance,r,i,z,I,K,K_err,flag,K_abs,r_abs,b,dist_to_group,brightest_K_nearby,SDSS_FRACGOOD,u-r,sersic_n,stellar_mass,N_sats_all_1,N_sats_removed_1,N_sats_rem1and2_1,N_SDSS_sats_1,N_sats_total_1,N_sats_all_2,N_sats_removed_2,N_sats_rem1and2_2,N_SDSS_sats_2,N_sats_total_2,skycoord,SAGA_name,n_qso_fuv_lt_19,n_qso_fuv_lt_18.5,n_qso_fuv_lt_18,n_uvqs_fuv_lt_19,n_uvqs_fuv_lt_18.5,n_uvqs_fuv_lt_18
,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,"deg,deg",,,,,,,
32,145.514,0.336452,27723,1912.35,30.0374,11.8626,11.603,11.4285,10.53,8.848,0.036,0,-23.5403,-20.5257,37.2791,8.62942,13.304,0.974112,1.63536,1.3137,16800000000.0,1,1,1,1,1,1,1,1,1,1,"145.514,0.336452",,0,0,0,0,0,0
13927,180.676,1.9768,38031,1961.14,29.5168,12.0011,11.5644,11.2205,10.98,8.743,0.027,0,-23.6073,-20.3492,62.2704,3.36159,11.815,0.98222,2.43461,3.12889,19800000000.0,0,0,0,0,0,0,0,0,0,0,"180.676,1.9768",Sopranos,1,1,1,0,0,0
16235,139.416,52.993,26246,2274.04,36.2141,12.9972,12.5874,12.277,14.99,9.406,0.02,0,-23.3884,-19.7972,43.0944,13.055,12.621,0.972638,2.35828,1.49728,12000000000.0,8,3,3,3,3,8,3,3,3,3,"139.416,52.993",,0,0,0,0,0,0
16559,154.888,58.2058,30176,2099.41,34.4951,12.3762,11.8854,11.586,11.85,9.469,0.031,0,-23.2198,-20.3126,49.2187,8.48479,11.148,0.98321,1.98864,1.75007,19100000000.0,4,2,3,3,3,4,2,3,3,3,"154.888,58.2058",,0,0,0,0,0,0
33446,123.241,36.2546,23028,2469.7,37.7379,12.3668,12.0137,11.739,11.77,9.407,0.032,0,-23.4769,-20.5171,31.306,14.5285,12.875,0.972509,2.04922,2.13367,21200000000.0,2,2,2,2,2,2,2,2,2,2,"123.241,36.2546",StarTrek,0,0,0,0,0,0
52773,193.841,58.3442,44025,2490.64,38.4771,12.406,12.05,11.79,11.39,9.047,0.023,0,-23.879,-20.52,58.7766,12.3121,11.077,0.986942,2.07612,1.75581,20300000000.0,0,0,0,0,0,0,0,0,0,0,"193.841,58.3442",,0,0,0,0,0,0
61945,354.131,0.297265,71883,2556.76,34.7674,12.0858,11.7031,11.4059,11.65,9.3,0.03,0,-23.4059,-20.6201,-57.1915,9.4085,12.75,0.988686,2.44616,6.0,24700000000.0,9,2,7,2,2,9,2,7,2,2,"354.131,0.297265",AnaK,0,0,0,0,0,0
63326,248.606,21.5409,58523,2936.5,41.3763,12.9283,12.5722,12.2773,--,9.588,0.027,0,-23.4958,-20.1555,39.299,15.3853,--,0.971828,1.96522,2.23754,13900000000.0,0,0,0,0,0,0,0,0,0,0,"248.606,21.5409",,0,0,0,0,0,0
69028,210.708,49.1726,50031,2012.51,31.7148,11.9989,11.5817,11.2854,11.28,8.789,0.027,0,-23.7173,-20.5074,64.0123,8.35729,10.968,0.98862,2.4304,1.78293,23000000000.0,4,3,3,3,3,4,3,3,3,3,"210.708,49.1726",,0,0,0,0,0,0


In [21]:
print('FUV<19 (JTQSO, UVQS)')
print(np.sum(tab_hosts['n_qso_fuv_lt_19']), np.sum(tab_hosts['n_uvqs_fuv_lt_19']),'\n')

print('FUV<18.5 (JTQSO, UVQS)')
print(np.sum(tab_hosts['n_qso_fuv_lt_18.5']), np.sum(tab_hosts['n_uvqs_fuv_lt_18.5']),'\n')

print('FUV<18 (JTQSO, UVQS)')
print(np.sum(tab_hosts['n_qso_fuv_lt_18']), np.sum(tab_hosts['n_uvqs_fuv_lt_18']))

FUV<19 (JTQSO, UVQS)
23 1 

FUV<18.5 (JTQSO, UVQS)
11 1 

FUV<18 (JTQSO, UVQS)
6 1


In [22]:
tab_saga_hosts = tab_hosts[tab_hosts['SAGA_name']!='']

print('FUV<19 (JTQSO, UVQS)')
print(np.sum(tab_saga_hosts['n_qso_fuv_lt_19']), np.sum(tab_saga_hosts['n_uvqs_fuv_lt_19']),'\n')

print('FUV<18.5 (JTQSO, UVQS)')
print(np.sum(tab_saga_hosts['n_qso_fuv_lt_18.5']), np.sum(tab_saga_hosts['n_uvqs_fuv_lt_18.5']),'\n')

print('FUV<18 (JTQSO, UVQS)')
print(np.sum(tab_saga_hosts['n_qso_fuv_lt_18']), np.sum(tab_saga_hosts['n_uvqs_fuv_lt_18']))

FUV<19 (JTQSO, UVQS)
6 0 

FUV<18.5 (JTQSO, UVQS)
4 0 

FUV<18 (JTQSO, UVQS)
4 0


In [23]:
tab_saga_hosts[tab_saga_hosts['n_qso_fuv_lt_19']>0].show_in_notebook()

NSAID,RA,Dec,PGC#,vhelio,distance,r,i,z,I,K,K_err,flag,K_abs,r_abs,b,dist_to_group,brightest_K_nearby,SDSS_FRACGOOD,u-r,sersic_n,stellar_mass,N_sats_all_1,N_sats_removed_1,N_sats_rem1and2_1,N_SDSS_sats_1,N_sats_total_1,N_sats_all_2,N_sats_removed_2,N_sats_rem1and2_2,N_SDSS_sats_2,N_sats_total_2,skycoord,SAGA_name,n_qso_fuv_lt_19,n_qso_fuv_lt_18.5,n_qso_fuv_lt_18,n_uvqs_fuv_lt_19,n_uvqs_fuv_lt_18.5,n_uvqs_fuv_lt_18
,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,"deg,deg",,,,,,,
13927,180.676,1.9768,38031,1961.14,29.5168,12.0011,11.5644,11.2205,10.98,8.743,0.027,0,-23.6073,-20.3492,62.2704,3.36159,11.815,0.98222,2.43461,3.12889,19800000000.0,0,0,0,0,0,0,0,0,0,0,"180.676,1.9768",Sopranos,1,1,1,0,0,0
85746,149.588,32.3699,28805,1451.33,25.1343,12.5439,12.2105,11.9101,11.27,8.899,0.016,0,-23.1023,-19.4575,52.3205,7.2153,--,0.978734,1.95676,0.932093,7680000000.0,1,1,1,0,1,1,1,1,0,1,"149.588,32.3699",MobyDick,3,2,2,0,0,0
140594,180.098,-1.10008,37845,1465.0,23.2487,11.1116,10.7406,10.4707,9.35,7.321,0.015,0,-24.511,-20.7204,59.2146,2.29212,12.669,0.985217,2.05631,1.7545,20600000000.0,2,1,2,1,1,2,1,2,1,1,"180.098,-1.10008",Alice,1,1,1,0,0,0
165536,221.546,-0.222942,52735,1659.36,25.3928,11.856,11.4773,11.2276,11.13,8.424,0.028,0,-23.5996,-20.1676,51.1661,3.0291,11.635,0.983643,2.34484,1.32237,16600000000.0,2,1,1,1,1,2,1,1,1,1,"221.546,-0.222942",Dune,1,0,0,0,0,0


In [24]:
matches_to_stack = []
for host, matches in zip(tab_hosts, host_matches19):
    if len(matches)>0:
        matches = matches.copy()
        del matches['skycoord']
        matches['SAGA_name'] = str(host['SAGA_name'])
        matches['SAGA_NSAID'] = host['NSAID']
        matches_to_stack.append(matches)
matched_qsos = table.vstack(matches_to_stack)
matched_qsos.sort('FUV_MAG')

matched_qsos.show_in_notebook()

SDSSNAME,RA,DEC,Z,PSFMAG_U,PSFMAGERR_U,PSFMAG_G,PSFMAGERR_G,PSFMAG_R,PSFMAGERR_R,PSFMAG_I,PSFMAGERR_I,PSFMAG_Z,PSFMAGERR_Z,A_U,LOGNHI,FIRSTMAG,FIRST_SN,FIRST_SEP,LG_RASS_RATE,RASS_SN,RASS_SEP,TWOMASSMAG_J,TWOMASSMAGERR_J,TWOMASSMAG_H,TWOMASSMAGERR_H,TWOMASSMAG_K,TWOMASSMAGERR_K,TWOMASS_SEP,M_I,DELTA_G_I,MORPHOLOGY,SCIENCEPRIMARY,MODE,UNIFORM,BESTPRIMTARGET,TS_B_QSO,TS_B_HIZ,TS_B_FIRST,TS_B_ROSAT,TS_B_SERENDIP,TS_B_STAR,TS_B_GAL,RUN_BEST,MJD_BEST,MJD_SPEC,PLATE,FIBERID,RERUN_BEST,CAMCOL_BEST,FIELD_BEST,OBJ_BEST,TARGPRIMTARGET,TS_T_QSO,TS_T_HIZ,TS_T_FIRST,TS_T_ROSAT,TS_T_SERENDIP,TS_T_STAR,TS_T_GAL,T_PSFMAG_U,T_PSFMAGERR_U,T_PSFMAG_G,T_PSFMAGERR_G,T_PSFMAG_R,T_PSFMAGERR_R,T_PSFMAG_I,T_PSFMAGERR_I,T_PSFMAG_Z,T_PSFMAGERR_Z,BESTOBJID,SPECOBJID,OBJECT_NAME,GALEX_OBJID,GALEX_RA,GALEX_DEC,GALEX_SEP,GALEX_TYPE,GALEX_BAND,NUV_FLUX,NUV_FLUXERR,FUV_FLUX,FUV_FLUXERR,NUV_MAG,NUV_MAGERR,FUV_MAG,FUV_MAGERR,GALEX_NOBJ,qso_to_host,qso_to_host_phys,SAGA_name,SAGA_NSAID
,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,deg,kpc,,
114758.59+271459.3,176.994140625,27.2498130798,0.3627,16.193,0.012,16.1,0.022,16.118,0.016,16.214,0.018,15.739,0.022,0.141,20.27,0.0,0.0,0.0,-0.776,5.563,8.56,15.241,0.048,14.679,0.066,13.893,0.043,0.226,-25.109,-0.318,0,1,1,1,7171,1,1,0,1,0,0,0,5112,53387,53793,2223,280,40,3,434,27,7171,1,1,0,1,0,0,0,16.186,0.012,16.103,0.022,16.118,0.016,16.209,0.018,15.714,0.022,587741708883722267,625949913577947136,SDSS J114758.59+271459.3,6374821310982262564,176.994182633,27.2495993357,0.78113603809,1,3,801.311,17.728,648.231,28.1976,16.6381,0.0240264,16.8736,0.0472402,1,0.347496949473,171.499647504,,140301
095820.94+322402.2,149.587265015,32.4006233215,0.5306,16.38,0.028,16.057,0.015,16.021,0.012,15.865,0.011,15.855,0.014,0.076,20.21,8.698,999.99,0.095,-9.0,0.0,0.0,14.945,0.039,14.443,0.05,13.76,0.042,0.032,-26.351,0.049,0,1,1,1,2097162,1,0,1,0,1,0,0,4518,53082,53433,1949,472,40,4,146,81,2097162,1,0,1,0,1,0,0,16.37,0.028,16.046,0.015,16.01,0.012,15.85,0.011,15.853,0.014,587739158191145041,548824224576307200,FBQS J095820.9+322402,6378410037478426750,149.587385979,32.400351098,1.04670804902,1,3,939.271,14.7886,508.15,16.6669,16.4656,0.0170988,17.1379,0.03562,1,0.0307295902541,13.4803426516,MobyDick,85746
095914.84+320357.2,149.81187439,32.0659103394,0.5639,17.196,0.012,16.882,0.021,16.992,0.013,16.931,0.016,17.033,0.018,0.076,20.225,0.0,0.0,0.0,-9.0,0.0,0.0,16.094,0.087,15.274,0.105,14.637,0.08,0.206,-25.433,-0.144,0,1,1,1,2,1,0,0,0,0,0,0,4518,53082,53433,1949,191,40,3,147,47,2,1,0,0,0,0,0,0,17.181,0.012,16.872,0.021,16.99,0.013,16.927,0.016,17.031,0.018,587739157654339631,548824223397707776,SDSS J095914.85+320357.2,6378410046068361930,149.811827774,32.0657063845,0.747883430811,1,3,575.656,15.8399,354.64,23.0186,16.9972,0.0298827,17.5284,0.070489,1,0.358166544482,157.119171001,MobyDick,85746
120233.08+022559.7,180.637863159,2.43326592445,0.273,17.299,0.061,17.322,0.016,17.27,0.023,17.339,0.026,16.899,0.023,0.142,20.272,0.0,0.0,0.0,-1.398,3.014,11.88,15.954,0.087,15.153,0.084,13.982,0.061,0.106,-23.308,-0.313,0,1,1,0,7171,1,1,0,1,0,0,0,1458,51668,52017,516,112,40,3,373,20,7173,1,1,0,1,0,0,0,17.29,0.011,17.295,0.018,17.261,0.012,17.264,0.017,16.853,0.036,587726015069224980,145464499766296576,SDSS J120233.08+022559.7,6379113808672066821,180.637932663,2.43328680621,0.261046830243,1,3,307.78,9.42953,315.524,18.6649,17.677,0.0332721,17.6553,0.0642425,2,0.458053924796,235.973508037,Sopranos,13927
092542.32+344108.6,141.426376343,34.6857299805,1.0667,16.649,0.018,16.516,0.017,16.325,0.011,16.314,0.014,16.24,0.018,0.115,20.188,0.0,0.0,0.0,-9.0,0.0,0.0,15.323,0.05,14.822,0.062,14.141,0.053,0.165,-27.612,-0.006,0,1,1,1,2,1,0,0,0,0,0,0,3704,52705,52991,1593,338,40,6,72,6,2,1,0,0,0,0,0,0,16.652,0.018,16.506,0.017,16.318,0.011,16.312,0.014,16.254,0.018,587735663156658182,448617233929732096,SDSS J092542.32+344108.6,6378445240104126593,141.426268426,34.6856815482,0.363941529562,1,3,602.578,8.4387,279.154,18.3079,

In [25]:
nms = ['{:.0f}kpc_{}'.format(row['qso_to_host_phys'], row['SAGA_name'] if row['SAGA_name']!='' else row['SAGA_NSAID']) for row in matched_qsos]
targeting.sampled_imagelist(matched_qsos, None, None, names=nms)

'name ra dec\n150kpc_Dune 221.693222046 -0.52883297205\n296kpc_144151 208.699386597 42.1115226746\n186kpc_162580 193.530136108 8.20126724243\n194kpc_165153 217.855407715 57.5743408203\n54kpc_165082 217.419769287 3.35718798637\n250kpc_142722 195.150909424 13.7849092484\n214kpc_144121 208.472167969 47.120967865\n137kpc_135440 130.496398926 14.1117200851\n49kpc_138201 162.815643311 28.0908641815\n155kpc_165707 222.964736938 59.2333564758\n13kpc_MobyDick 149.587265015 32.4006233215\n258kpc_162547 193.815505981 9.80721187592\n224kpc_138201 163.197784424 28.3861732483\n236kpc_Sopranos 180.637863159 2.43326592445\n268kpc_144953 215.80909729 50.9270248413\n162kpc_135879 141.426376343 34.6857299805\n107kpc_158541 159.188140869 37.1170272827\n261kpc_Alice 180.611495972 -1.48757696152\n171kpc_140301 176.994140625 27.2498130798\n233kpc_94217 209.111801147 28.7937984467\n244kpc_165082 217.941635132 3.02884888649\n157kpc_MobyDick 149.81187439 32.0659103394\n209kpc_MobyDick 149.096725464 32.606018066

# X-match sats 

In [26]:
def get_sat_closest(sattab, qsotab, sorton=None):
    idx, d2d, _ = sattab['skycoord'].match_to_catalog_sky(qsotab['skycoord'], nthneighbor=1)
    tab = qsotab[idx]
    tab['match_d2d'] = d2d
    tab['match_d2d_phys'] = (d2d.radian*sattab['HOST_DIST']*u.Mpc).to(u.kpc)
    
    if sorton is not None:
        #trick to make sort work
        sorti = np.argsort(tab[sorton])
        tab = tab[sorti]
        sattab = sattab[sorti]
    return table.hstack((sattab, tab))

In [27]:
sat_matches = get_sat_closest(tab_sats, tab_qso19, sorton='match_d2d_phys')
sat_matches_uvqs = get_sat_closest(tab_sats, uvqs_good_tab, sorton='match_d2d_phys')

In [28]:
sat_matches['RA_1', 'DEC_1', 'RA_2', 'DEC_2', 'TELNAME', 'HOST_SAGA_NAME', 'match_d2d_phys'].show_in_notebook()

RA_1,DEC_1,RA_2,DEC_2,TELNAME,HOST_SAGA_NAME,match_d2d_phys
,,,,,,kpc
162.849821529,28.1120278164,162.815643311,28.0908641815,NSA,,15.571269356
141.506228227,34.6534021854,141.426376343,34.6857299805,NSA,,35.3521428813
162.705990064,28.0587029336,162.815643311,28.0908641815,NSA,,43.0991415677
195.105563951,13.6705858419,195.150909424,13.7849092484,NSA,,64.2071453789
162.816110258,27.8489705993,162.815643311,28.0908641815,NSA,,102.253944422
193.556579042,8.3577808181,193.530136108,8.20126724243,NSA,,110.849879259
217.266218915,3.14899055549,217.419769287,3.35718798637,NSA,,112.425049336
217.818940474,3.25673766255,217.941635132,3.02884888649,NSA,,112.50412402
141.141087343,34.6588051715,141.426376343,34.6857299805,NSA,,114.05850752


In [29]:
nms = ['SAT_{}_{:.0f}kpc'.format(i, row['match_d2d_phys']) for i, row in enumerate(sat_matches)]
targeting.sampled_imagelist(sat_matches['RA_1'], sat_matches['DEC_1'], n=len(sat_matches), names=nms)

'name ra dec\nSAT_0_16kpc 162.849821529 28.1120278164\nSAT_1_35kpc 141.506228227 34.6534021854\nSAT_2_43kpc 162.705990064 28.0587029336\nSAT_3_64kpc 195.105563951 13.6705858419\nSAT_4_102kpc 162.816110258 27.8489705993\nSAT_5_111kpc 193.556579042 8.3577808181\nSAT_6_112kpc 217.266218915 3.14899055549\nSAT_7_113kpc 217.818940474 3.25673766255\nSAT_8_114kpc 141.141087343 34.6588051715\nSAT_9_118kpc 162.904734277 27.822117387\nSAT_10_139kpc 180.419076868 -1.20276655787\nSAT_11_143kpc 130.700889417 14.2652497454\nSAT_12_145kpc 149.687168865 32.7202654623\nSAT_13_150kpc 180.29544216 -1.2976838319\nSAT_14_153kpc 217.970758535 3.38004139202\nSAT_15_162kpc 141.021441102 34.6469717218\nSAT_16_164kpc 222.900268358 58.9548243013\nSAT_17_166kpc 193.358416186 8.03555517632\nSAT_18_183kpc 154.228523279 58.394558724\nSAT_19_185kpc 217.507687769 3.77289260119\nSAT_20_191kpc 130.611942997 14.4258881655\nSAT_21_198kpc 141.450424263 34.2768708872\nSAT_22_204kpc 162.303384997 27.922597647\nSAT_23_213kpc 1

In [30]:
nms = ['QSO_{}_{:.0f}kpc'.format(i, row['match_d2d_phys']) for i, row in enumerate(sat_matches)]
targeting.sampled_imagelist(sat_matches['RA_2'], sat_matches['DEC_2'], n=len(sat_matches), names=nms)

'name ra dec\nQSO_0_16kpc 162.815643311 28.0908641815\nQSO_1_35kpc 141.426376343 34.6857299805\nQSO_2_43kpc 162.815643311 28.0908641815\nQSO_3_64kpc 195.150909424 13.7849092484\nQSO_4_102kpc 162.815643311 28.0908641815\nQSO_5_111kpc 193.530136108 8.20126724243\nQSO_6_112kpc 217.419769287 3.35718798637\nQSO_7_113kpc 217.941635132 3.02884888649\nQSO_8_114kpc 141.426376343 34.6857299805\nQSO_9_118kpc 162.815643311 28.0908641815\nQSO_10_139kpc 180.611495972 -1.48757696152\nQSO_11_143kpc 130.496398926 14.1117200851\nQSO_12_145kpc 149.587265015 32.4006233215\nQSO_13_150kpc 180.611495972 -1.48757696152\nQSO_14_153kpc 217.941635132 3.02884888649\nQSO_15_162kpc 141.426376343 34.6857299805\nQSO_16_164kpc 222.964736938 59.2333564758\nQSO_17_166kpc 193.530136108 8.20126724243\nQSO_18_183kpc 153.679718018 58.2953834534\nQSO_19_185kpc 217.419769287 3.35718798637\nQSO_20_191kpc 130.496398926 14.1117200851\nQSO_21_198kpc 141.426376343 34.6857299805\nQSO_22_204kpc 162.815643311 28.0908641815\nQSO_23_21